# Analyze FMCIB features for original and READII negative control CT images

This notebook is set up to use outputs from the `run_fmcib.ipynb` notebook.

Image features extracted from CT images cropped to a Gross Tumour Volume (GTV) undergo correlation analysis. Results are compared across READII negative control image types.

## Set up pixi environment kernel
If you ran `run_readii_prep`, the kernel should already exist and you can skip to step 2.

1. Run the following commands in the terminal:

    ```bash
    $ pixi install -e default

    $ pixi run kernel
    ```

2. In the `Select Kernel` menu at the top right of the notebook, select `Jupyter Kernel` as the source. 

3. Refresh the options and one called `readii-fmcib` should appear. Select this option.

## Imports

In [ ]:
import itertools
import pandas as pd
import numpy as np

from pathlib import Path

from readii.io.loaders import loadImageDatasetConfig, loadFeatureFilesFromImageTypes
from readii.io.writers.correlation_writer import CorrelationWriter
from readii.analyze.correlation import getFeatureCorrelations, getSelfCorrelations

from sklearn.cluster import AgglomerativeClustering 

import sys; sys.path.append("code")
from analyze import prepPatientIndex, makeAllHeatmapPlots, makeAllHistogramPlots, sortCorrelationsByClustering, makeAllClusterHeatmapPlots

## Initialize dataset name and load config 

In [2]:
config = loadImageDatasetConfig("RADCURE", Path("config"))

DATASET_NAME = config["dataset_name"]

PAT_ID_PATTERN = config['patient_id_pattern']

NEG_CONTROL_REGIONS = config["negative_control_regions"]
NEG_CONTROL_TYPES = config["negative_control_types"]

CROP_METHOD = config['crop_method']

# Get full list of image types to run FMCIB on
negative_control_list = [f"{negative_control[0]}_{negative_control[1]}" for negative_control in itertools.product(NEG_CONTROL_TYPES, NEG_CONTROL_REGIONS)]

## Set up data directories

In [3]:
# Set features input directory
features_dir = Path("results", DATASET_NAME, "fmcib_features", f"cropped_{CROP_METHOD}")

# Make correlation results output directory
correlations_dir = Path("results", DATASET_NAME, "analysis", "correlations", f"cropped_{CROP_METHOD}")
for combo in itertools.product([correlations_dir], ["matrix", "heatmap", "histogram", "clustered"]):
    Path(*combo).mkdir(parents=True, exist_ok=True)

## Load all extracted feature sets

In [4]:
# Load the extracted feature data
# This makes a dictionary of feature sets, with the image type as the keys
extracted_feature_sets = loadFeatureFilesFromImageTypes(extracted_feature_dir=features_dir,
                                                        image_types = (["original"] + negative_control_list), 
                                                        drop_labels = False)

# Run correlation analysis for each image type

In [5]:
# Set up writers for correlation and plots
corr_matrix_writer = CorrelationWriter(
    root_directory = correlations_dir / "matrix",
    filename_format = DATASET_NAME + "_{VerticalFeatureType}_{HorizontalFeatureType}_{CorrelationType}_correlations.csv",
    overwrite = False,
    create_dirs = True
)

In [ ]:
# Name of the column used to extract the patient ID for a row of features
file_path_column = 'image_path'

# Correlation method to apply
correlation_method = "pearson"

# Colormap to use for plots
heatmap_cmap = "nipy_spectral"

# initialize clustering id array
clustering = np.array([])

# Whether to overwrite existing files
overwrite = True

# Get and set up the feature dataframe for the original features once
vertical_feature_type = "original"
vertical_features_df = prepPatientIndex(extracted_feature_sets[vertical_feature_type],
                                        file_path_column,
                                        PAT_ID_PATTERN)

# Iterate over each negative control feature set and perform correlation analysis
for horizontal_feature_type in negative_control_list:
    print(f"Processing {vertical_feature_type} vs {horizontal_feature_type} correlations.")

    # Generate the output path for matrix file existence check
    corr_matrix_output_path = corr_matrix_writer.resolve_path(VerticalFeatureType=vertical_feature_type,
                                                              HorizontalFeatureType=horizontal_feature_type,
                                                              CorrelationType=correlation_method)
    
    # Get extracted features for this image type, extract set the patient ID as the dataframe index, remove image_path column
    horizontal_features_df = prepPatientIndex(extracted_feature_sets[horizontal_feature_type], 
                                              file_path_column = file_path_column, 
                                              pat_id_pattern = PAT_ID_PATTERN)

    
    # Load existing correlation matrix if it's available
    if corr_matrix_output_path.exists() and corr_matrix_output_path.is_file():
        print("Loading correlation matrix.")
        feature_correlation_df = pd.read_csv(corr_matrix_output_path, index_col=0)
    
    # Calculate the correlation matrix if the file doesn't exist
    else:
        print("Calculating correlation matrix.")
        # Calculate correlations between original image features and image type features
        feature_correlation_df = getFeatureCorrelations(vertical_features=vertical_features_df,
                                                        horizontal_features=horizontal_features_df,
                                                        vertical_feature_name=vertical_feature_type,
                                                        horizontal_feature_name=horizontal_feature_type,
                                                        method = correlation_method)
        # save out the correlation dataframe
        corr_matrix_writer.save(feature_correlation_df, 
                                VerticalFeatureType=vertical_feature_type,
                                HorizontalFeatureType=horizontal_feature_type,
                                CorrelationType=correlation_method)
        
    print("Generating heatmaps for correlations.")
    vert_heatmap_path, horiz_heatmap_path, cross_heatmap_path = makeAllHeatmapPlots(feature_correlation_df,
                                                                                    vertical_feature_type,
                                                                                    horizontal_feature_type,
                                                                                    correlations_dir,
                                                                                    correlation_method,
                                                                                    heatmap_cmap,
                                                                                    overwrite)
    
    # print("Generating histograms for correlations.")
    # vert_histogram_path, horiz_histogram_path, cross_histogram_path = makeAllHistogramPlots(feature_correlation_df,
    #                                                                                         vertical_feature_type,
    #                                                                                         horizontal_feature_type,
    #                                                                                         correlations_dir,
    #                                                                                         correlation_method,
    #                                                                                         num_bins=450,
    #                                                                                         self_corr_y_max = 250000,
    #                                                                                         cross_corr_y_max = 950000,
    #                                                                                         overwrite=overwrite)
    print("Generating heatmaps for clustered correlations.")
    if len(clustering) == 0:
        # Cluster the features based on the correlations from the Original image
        original_corr = getSelfCorrelations(feature_correlation_df, vertical_feature_type)
        clustering = AgglomerativeClustering(linkage="complete", metric="precomputed", n_clusters = None, distance_threshold = 0).fit_predict(original_corr)
    
    
    vert_heatmap_path, horiz_heatmap_path, cross_heatmap_path = makeAllClusterHeatmapPlots(feature_correlation_df,
                                                                                           vertical_feature_type,
                                                                                           horizontal_feature_type,
                                                                                           clustering,
                                                                                           correlations_dir / "clustered",
                                                                                           correlation_method,
                                                                                           heatmap_cmap,
                                                                                           overwrite)


Processing original vs shuffled_full correlations.
     Loading correlation matrix.
     Generating histograms for correlations.
Plotting vertical feature correlations histogram...


2025-01-15T13:14:44-0500 [warning  ] File results/RADCURE/analysis/correlations/cropped_bbox/histogram/original_pearson_self_correlation_histogram.png already exists. 
Set PlotWriter.overwrite to True to overwrite. [readii] call=plot_correlation.saveCorrelationHistogram:132


Plotting horizontal feature correlations histogram...


2025-01-15T13:14:45-0500 [warning  ] File results/RADCURE/analysis/correlations/cropped_bbox/histogram/shuffled_full_pearson_self_correlation_histogram.png already exists. 
Set PlotWriter.overwrite to True to overwrite. [readii] call=plot_correlation.saveCorrelationHistogram:132


Plotting cross feature correlations histogram...


...

2025-01-15T13:16:34-0500 [warning  ] File results/RADCURE/analysis/correlations/cropped_bbox/histogram/original_pearson_self_correlation_histogram.png already exists. 
Set PlotWriter.overwrite to True to overwrite. [readii] call=plot_correlation.saveCorrelationHistogram:132


Plotting horizontal feature correlations histogram...


2025-01-15T13:16:35-0500 [warning  ] File results/RADCURE/analysis/correlations/cropped_bbox/histogram/randomized_sampled_non_roi_pearson_self_correlation_histogram.png already exists. 
Set PlotWriter.overwrite to True to overwrite. [readii] call=plot_correlation.saveCorrelationHistogram:132


Plotting cross feature correlations histogram...


2025-01-15T13:16:35-0500 [warning  ] File results/RADCURE/analysis/correlations/cropped_bbox/histogram/original_vs_randomized_sampled_non_roi_pearson_cross_correlation_histogram.png already exists. Overwriting. [readii] call=plot_writer.save:92


# Original Correlations Only
Run this section if no negative controls are being run and you want to analyze just the original image features.

In [ ]:
from readii.analyze.plot_correlation import plotSelfCorrHeatmap, plotSelfCorrHistogram

# Name of the column used to extract the patient ID for a row of features
file_path_column = 'image_path'

# Correlation method to apply
correlation_method = "pearson"

# Colormap to use for plots
heatmap_cmap = "nipy_spectral"

# Whether to overwrite existing files
overwrite = True

# Get and set up the feature dataframe for the original features once
vertical_feature_type = "original"
vertical_features_df = prepPatientIndex(extracted_feature_sets[vertical_feature_type],
                                        file_path_column,
                                        PAT_ID_PATTERN)

# Generate the output path for matrix file existence check
corr_matrix_output_path = corr_matrix_writer.resolve_path(VerticalFeatureType=vertical_feature_type,
                                                          HorizontalFeatureType=vertical_feature_type,
                                                          CorrelationType=correlation_method)

# Load existing correlation matrix if it's available
if corr_matrix_output_path.exists() and corr_matrix_output_path.is_file():
    print("Loading correlation matrix.")
    vertical_self_corr_df = pd.read_csv(corr_matrix_output_path, index_col=0)

# Calculate the correlation matrix if the file doesn't exist
else:
    print("Calculating correlation matrix.")
    # Calculate correlations between vertical image features 
    vertical_self_corr_df = vertical_features_df.corr(method = correlation_method)
    # Add feature type suffix to column names and index for the plotting functions to work
    vertical_self_corr_df = vertical_self_corr_df.add_suffix(f"_{vertical_feature_type}", 0)
    vertical_self_corr_df = vertical_self_corr_df.add_suffix(f"_{vertical_feature_type}", 1)

    # save out the correlation dataframe
    corr_matrix_writer.save(vertical_self_corr_df, 
                            VerticalFeatureType=vertical_feature_type,
                            HorizontalFeatureType=vertical_feature_type,
                            CorrelationType=correlation_method)
    
# Make plots
_, vert_heatmap_path = plotSelfCorrHeatmap(vertical_self_corr_df,
                                           vertical_feature_type,
                                           correlation_method,
                                           heatmap_cmap,
                                           correlations_dir,
                                           overwrite)
    
_, vert_histogram_path = plotSelfCorrHistogram(vertical_self_corr_df,
                                               vertical_feature_type,
                                               correlation_method,
                                               num_bins=450,
                                               y_upper_bound = 100000,
                                               save_dir_path=correlations_dir,
                                               overwrite=overwrite)

# Clustering Analysis

In [10]:
# Name of the column used to extract the patient ID for a row of features
file_path_column = 'image_path'

# Correlation method to apply
correlation_method = "pearson"

# Colormap to use for plots
heatmap_cmap = "nipy_spectral"

# Whether to overwrite existing files
overwrite = False

# Get and set up the feature dataframe for the original features once
vertical_feature_type = "original"

clustering = np.array([])

output_fig_directory = correlations_dir / "clustered"

# Iterate over each negative control feature set and perform correlation analysis
for horizontal_feature_type in negative_control_list:
    print(f"Processing {vertical_feature_type} vs {horizontal_feature_type} correlations.")

    feature_correlation_df = pd.read_csv(f"results/RADCURE/analysis/correlations/cropped_centroid/matrix/RADCURE_{vertical_feature_type}_{horizontal_feature_type}_pearson_correlations.csv", index_col=0)

    if len(clustering) == 0:
        # Cluster the features based on the correlations from the Original image
        original_corr = getSelfCorrelations(feature_correlation_df, vertical_feature_type)
        clustering = AgglomerativeClustering(linkage="complete", metric="precomputed", n_clusters = None, distance_threshold = 0).fit_predict(original_corr)
    
    print("     Generating heatmaps for clustered correlations.")
    vert_heatmap_path, horiz_heatmap_path, cross_heatmap_path = makeAllClusterHeatmapPlots(feature_correlation_df,
                                                                                           vertical_feature_type,
                                                                                           horizontal_feature_type,
                                                                                           clustering,
                                                                                           output_fig_directory,
                                                                                           correlation_method,
                                                                                           heatmap_cmap,
                                                                                           overwrite)

Processing original vs shuffled_full correlations.
     Generating heatmaps for clustered correlations.
Plotting vertical feature correlations heatmap...


2025-01-30T12:26:36-0500 [warning  ] File results/RADCURE/analysis/correlations/cropped_bbox/clustered/heatmap/nipy_spectral/original_pearson_self_correlation_heatmap.png already exists. 
Set PlotWriter.overwrite to True to overwrite. [readii] call=plot_correlation.saveCorrelationHeatmap:73


Plotting horizontal feature correlations heatmap...
Plotting cross feature correlations heatmap...
Processing original vs shuffled_roi correlations.
     Generating heatmaps for clustered correlations.
Plotting vertical feature correlations heatmap...


2025-01-30T12:27:36-0500 [warning  ] File results/RADCURE/analysis/correlations/cropped_bbox/clustered/heatmap/nipy_spectral/original_pearson_self_correlation_heatmap.png already exists. 
Set PlotWriter.overwrite to True to overwrite. [readii] call=plot_correlation.saveCorrelationHeatmap:73


Plotting horizontal feature correlations heatmap...
Plotting cross feature correlations heatmap...
Processing original vs shuffled_non_roi correlations.
     Generating heatmaps for clustered correlations.
Plotting vertical feature correlations heatmap...


...

2025-01-30T12:31:44-0500 [warning  ] File results/RADCURE/analysis/correlations/cropped_bbox/clustered/heatmap/nipy_spectral/original_pearson_self_correlation_heatmap.png already exists. 
Set PlotWriter.overwrite to True to overwrite. [readii] call=plot_correlation.saveCorrelationHeatmap:73


Plotting horizontal feature correlations heatmap...


2025-01-30T12:31:54-0500 [warning  ] File results/RADCURE/analysis/correlations/cropped_bbox/clustered/heatmap/nipy_spectral/randomized_sampled_non_roi_pearson_self_correlation_heatmap.png already exists. 
Set PlotWriter.overwrite to True to overwrite. [readii] call=plot_correlation.saveCorrelationHeatmap:73


Plotting cross feature correlations heatmap...


2025-01-30T12:32:02-0500 [warning  ] File results/RADCURE/analysis/correlations/cropped_bbox/clustered/heatmap/nipy_spectral/original_vs_randomized_sampled_non_roi_pearson_cross_correlation_heatmap.png already exists. 
Set PlotWriter.overwrite to True to overwrite. [readii] call=plot_correlation.saveCorrelationHeatmap:73
